In [1]:
import pandas as pd
import numpy as np
import os
import geopandas as gp

In [2]:
# -- read the inspection files
try:
    inspection
except:
    print("reading PIP_InspectionMain.xlsx...")
    in_path = os.path.join('../../data','PIP')
    in_name = os.path.join(in_path,'PIP_InspectionMain.xlsx')
    inspection  = pd.read_excel(in_name)

reading PIP_InspectionMain.xlsx...


In [3]:
# -- get all the ratings information
try:
    rating
except:
    print("reading PIP_ALLSITES.xlsx...")
    as_path  = os.path.join('../../data','PIP')
    as_name  = os.path.join(as_path,'PIP_FeatureRatings.xlsx')
    rating = pd.read_excel(as_name)
    
    # -- Sustitute numerical values to ratings
    rating.loc[rating['Rating'] == 'a', ['Rating']] = 1
    rating.loc[rating['Rating'] == 'A', ['Rating']] = 1
    rating.loc[rating['Rating'] == 'N', ['Rating']] = np.nan
    rating.loc[rating['Rating'] == 'U', ['Rating']] = 0
    rating.loc[rating['Rating'] == 'U/S', ['Rating']] = 0

reading PIP_ALLSITES.xlsx...


In [4]:
# -- get all the sites information
try:
    sites
except:
    print("reading PIP_ALLSITES.xlsx...")
    as_path  = os.path.join('../../data','PIP')
    as_name  = os.path.join(as_path,'PIP_ALLSITES.xlsx')
    sites = pd.read_excel(as_name)

reading PIP_ALLSITES.xlsx...


In [5]:
# -- read in the property files
try:
    prop
except:
    print("reading Property.shp...")
    pr_path = os.path.join('../../shapeData','CUSPExportShps')
    pr_name = os.path.join(pr_path,'Property.shp')
    prop    = gp.GeoDataFrame.from_file(pr_name)

reading Property.shp...


In [14]:
# -- pull off only first zip in list (if there are multiple)
prop.ZIPCODE = prop.ZIPCODE.apply(lambda x: x[:5])

In [15]:
# -- Change Rating type to float for calculations
rating[['Rating']] = rating[['Rating']].astype(float)

In [16]:
# -- Create pivot table of inspections.
pivotInspect = rating.pivot(index='Inspection ID', columns = 'Feature', values = 'Rating')

In [35]:
# -- create the base ID column  MAY HAVE TO BE TWEAKED
inspection['PID_base'] = [i.split('-')[0].replace('Z','') 
                      for i in inspection['Prop ID']]

In [ ]:
######################## Checking the unmatching Identifiers ########################

In [117]:
# -- Create a list of unique Porperty IDs in the Inspection Dataframe
InspectionUniqueId = inspection['PID_base'].unique()

In [141]:
# -- Create a dataframe of Unique Prop Ids in Sites file
sitesPropIdDF = pd.DataFrame(sites['PropNum'].unique())
sitesPropIdDF.columns = ['propID']

In [142]:
# -- Extract the unmatching prop IDs between AllSites and Inspections
UMsitesPropIdDF=sitesPropIdDF[~sitesPropIdDF.propID.isin(InspectionUniqueId)]

In [143]:
# -- export file
UMsitesPropIdDF.to_csv('../Outputs/UnmatchingAllSitesInspectionsMain.csv', sep='\t', encoding='utf-8')

In [120]:
# -- Create a list of unique Inspection IDs in the Rating Dataframe
UniqeRatingId = rating['Inspection ID'].unique()

In [121]:
# -- Create a dataframe of Inspection IDs in Inspection file
InspecUniqueDF = pd.DataFrame(inspection['Inspection ID'].unique())
InspecUniqueDF.columns = ['InspectionID']

In [139]:
# -- Extract the unmatching Inspection IDs between AllSites and Inspections
UMInspecUniqueDF = InspecUniqueDF[~InspecUniqueDF.InspectionID.isin(UniqeRatingId)]

In [140]:
# -- export file
UMShapeFilePropDF.to_csv('../Outputs/UnmatchingRatingInspectionsMain.csv', sep='\t', encoding='utf-8')

In [135]:
# -- Extract the unmatching prop IDs between Prop Shapefile and Inspection
ShapeFilePropDF = prop[['GISPROPNUM']]
UMShapeFilePropDF = ShapeFilePropDF[~ShapeFilePropDF.GISPROPNUM.isin(InspectionUniqueId)]

In [137]:
# -- export file
UMShapeFilePropDF.to_csv('../Outputs/UnmatchingPropShapeFileInspectionsMain.csv', sep='\t', encoding='utf-8')

In [60]:
# -- Remove Greenstreets from sites
# sites = sites[sites['Category'] != 'Greenstreet']

In [11]:
# -- Merge inspection report with sites [Greenstreets are removed from sites]
# finalDF = pd.merge(sites, inspection, on = 'Prop ID', how = 'left')

In [12]:
# -- Drop sites that have inspections
# finalDF = finalDF[finalDF['Inspection ID'].notnull()]

In [13]:
# -- Merge frame with pivot table of inspections on Inspection ID
# finalDF = pd.merge(finalDF, pivotInspect, left_on='Inspection ID', right_index = True, how='left')

In [14]:
##### -- Merge the inspection and properties information
# finalDF = pd.merge(finalDF, prop, 'left', left_on='PID_base', 
#                   right_on='GISPROPNUM')

In [21]:
# -- check if there are inspected parks that aren't in allsites
check_ALLSITES = False
if check_ALLSITES:
    print("checking allsites file...")
    for ii,pid in enumerate(inspection['Prop ID']):
        flag = True
        for asid in sites['Prop ID']:
            if pid==asid:
                flag = False
                break
        if flag:
            print("couldn't find {0} : {1}".format(ii,pid))

In [65]:
# -- check features file
check_FEATRAT = False
if check_FEATRAT:
    for ii,iid in enumerate(inspection['Inspection ID']):
        if iid not in pivotInspect.index:
            print("couldn't find {0} : Inspection ID: {1}".format(ii,iid))

In [66]:
# -- check properties
check_PROP = False
if check_PROP:
    print("checking properties file...")
    pid_bad = []
    pid_bad_ii = []
    for ii,pid in enumerate(inspection['PID_base']):
        flag = True
        for gpn in prop.GISPROPNUM:
            if pid==gpn:
                flag = False
                break
        if flag:
            if pid not in pid_bad:
                pid_bad_ii.append(ii)
                pid_bad.append(pid)
            print("couldn't find {0} : {1}".format(ii,pid))

    for ii in range(len(pid_bad_ii)):
        subcat = sites.iloc[sites[sites['Prop ID'] == \
                                inspection.iloc[pid_bad_ii[ii]]['Prop ID']] \
                                .index[0]]['Sub-Category']
        print("{0:8} : {1}" \
                  .format(inspection.iloc[pid_bad_ii[ii]]['Prop ID'],subcat))